# Análise e Previsão de Sucesso de Startups

Este notebook apresenta uma análise completa para prever o sucesso ou fracasso de startups com base em seus dados históricos, incluindo informações sobre investimentos, localização e características operacionais.

## 1. Configuração do Ambiente

Nesta seção, importamos todas as bibliotecas necessárias e configuramos o ambiente de trabalho.

In [ ]:
# Importação das bibliotecas necessárias
import warnings
warnings.filterwarnings('ignore')

# Bibliotecas para análise de dados e visualização
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Bibliotecas para machine learning
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, VotingClassifier
from sklearn.feature_selection import SelectFromModel, SelectKBest, mutual_info_classif
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Configuração da semente aleatória para reprodutibilidade
RANDOM_STATE = 42

# Configurações de visualização
plt.style.use('seaborn')
sns.set_palette('viridis')
pd.set_option('display.max_columns', None)

## 2. Carregamento e Exploração Inicial dos Dados

Nesta seção, vamos carregar os dados e fazer uma exploração inicial para entender sua estrutura e características principais.

In [ ]:
# Carregamento dos datasets de treino e teste
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

# Exibindo informações básicas sobre os datasets
print("=== Dataset de Treino ===")
print(f"Dimensões: {train_df.shape}")
print("\nPrimeiras linhas:")
display(train_df.head())
print("\nInformações sobre as colunas:")
print(train_df.info())

print("\n=== Dataset de Teste ===")
print(f"Dimensões: {test_df.shape}")
print("\nPrimeiras linhas:")
display(test_df.head())

### 2.1 Análise da Variável Alvo

Vamos analisar a distribuição da nossa variável alvo (sucesso/insucesso) para entender o balanceamento das classes.

In [ ]:
# Análise da distribuição da variável alvo
labels_dist = train_df['labels'].value_counts()
labels_pct = train_df['labels'].value_counts(normalize=True)

# Criando um gráfico de barras para visualizar a distribuição
plt.figure(figsize=(10, 6))
sns.countplot(data=train_df, x='labels')
plt.title('Distribuição da Variável Alvo')
plt.xlabel('Rótulo (0=Insucesso, 1=Sucesso)')
plt.ylabel('Quantidade')

# Adicionando as porcentagens nas barras
total = len(train_df['labels'])
for i, v in enumerate(labels_dist):
    plt.text(i, v, f'{labels_pct[i]:.1%}', ha='center', va='bottom')

plt.show()

print("\nDistribuição detalhada:")
print(f"Sucesso (1): {labels_dist[1]} casos ({labels_pct[1]:.1%})")
print(f"Insucesso (0): {labels_dist[0]} casos ({labels_pct[0]:.1%})")
print(f"\nRazão de desbalanceamento: {max(labels_pct)/min(labels_pct):.2f}:1")

### 2.2 Análise de Valores Ausentes

Vamos verificar se existem valores ausentes nos nossos dados e visualizar sua distribuição.

In [ ]:
# Análise de valores ausentes nos datasets de treino e teste
def analyze_missing_values(df, title):
    missing = df.isnull().sum()
    missing_pct = (df.isnull().sum() / len(df)) * 100
    missing_df = pd.DataFrame({
        'Valores Ausentes': missing,
        'Porcentagem (%)': missing_pct
    })
    missing_df = missing_df[missing_df['Valores Ausentes'] > 0].sort_values('Valores Ausentes', ascending=False)
    
    if len(missing_df) > 0:
        plt.figure(figsize=(12, 6))
        plt.barh(y=missing_df.index, width=missing_df['Porcentagem (%)'])
        plt.title(f'Porcentagem de Valores Ausentes - {title}')
        plt.xlabel('Porcentagem de Valores Ausentes')
        plt.tight_layout()
        plt.show()
        
        print(f"\nDetalhamento de valores ausentes - {title}:")
        print(missing_df)
    else:
        print(f"\n{title}: Não foram encontrados valores ausentes!")

# Análise para o dataset de treino
analyze_missing_values(train_df, "Dataset de Treino")

# Análise para o dataset de teste
analyze_missing_values(test_df, "Dataset de Teste")

### 2.3 Análise de Correlações

Vamos analisar as correlações entre as variáveis numéricas e identificar padrões importantes.

In [ ]:
# Selecionando apenas as colunas numéricas
numeric_cols = train_df.select_dtypes(include=[np.number]).columns.drop(['id', 'labels'], errors='ignore')

# Calculando a matriz de correlação
corr_matrix = train_df[numeric_cols].corr()

# Criando o heatmap
plt.figure(figsize=(15, 12))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            fmt='.2f', square=True, linewidths=0.5)
plt.title('Matriz de Correlação - Features Numéricas')
plt.tight_layout()
plt.show()

# Analisando correlações com a variável alvo
target_corr = corr_matrix['labels'].sort_values(ascending=False)
print("\nTop 10 features mais correlacionadas com o sucesso:")
print(target_corr.head(10))
print("\nTop 10 features mais correlacionadas com o insucesso:")
print(target_corr.tail(10))

## 3. Pré-processamento dos Dados

Nesta seção, vamos realizar as seguintes etapas:
1. Tratamento de valores ausentes
2. Tratamento de outliers
3. Engenharia de features
4. Normalização dos dados
5. Codificação de variáveis categóricas

### 3.1 Tratamento de Valores Ausentes

In [ ]:
# Criando uma função para tratar valores ausentes
def handle_missing_values(train_df, test_df):
    # Criando cópias para não modificar os dataframes originais
    train = train_df.copy()
    test = test_df.copy()
    
    # Identificando colunas numéricas
    numeric_cols = train.select_dtypes(include=[np.number]).columns.drop(['id', 'labels'], errors='ignore')
    
    # Usando SimpleImputer para preencher valores ausentes com a mediana
    imputer = SimpleImputer(strategy='median')
    
    # Aplicando o imputer nas colunas numéricas
    train[numeric_cols] = imputer.fit_transform(train[numeric_cols])
    test[numeric_cols] = imputer.transform(test[numeric_cols])
    
    return train, test

# Aplicando o tratamento de valores ausentes
train_clean, test_clean = handle_missing_values(train_df, test_df)

print("Verificando valores ausentes após o tratamento:")
print("\nDataset de Treino:")
print(train_clean.isnull().sum().sum(), "valores ausentes")
print("\nDataset de Teste:")
print(test_clean.isnull().sum().sum(), "valores ausentes")

### 3.2 Tratamento de Outliers

Vamos identificar e tratar outliers usando o método de capping nos percentis.

In [ ]:
# Função para tratar outliers usando o método de capping
def handle_outliers(train_df, test_df, lower_percentile=1, upper_percentile=99):
    train = train_df.copy()
    test = test_df.copy()
    
    numeric_cols = train.select_dtypes(include=[np.number]).columns.drop(['id', 'labels'], errors='ignore')
    
    for col in numeric_cols:
        # Calculando os limites baseados nos percentis do conjunto de treino
        lower_limit = np.percentile(train[col], lower_percentile)
        upper_limit = np.percentile(train[col], upper_percentile)
        
        # Aplicando o capping em ambos os conjuntos
        train[col] = train[col].clip(lower=lower_limit, upper=upper_limit)
        test[col] = test[col].clip(lower=lower_limit, upper=upper_limit)
    
    return train, test

# Aplicando o tratamento de outliers
train_clean, test_clean = handle_outliers(train_clean, test_clean)

# Visualizando a distribuição antes e depois para algumas features importantes
features_to_check = ['funding_total_usd', 'age_first_funding_year', 'relationships']

fig, axes = plt.subplots(len(features_to_check), 2, figsize=(15, 5*len(features_to_check)))
for idx, feature in enumerate(features_to_check):
    if feature in train_df.columns:
        # Antes do tratamento
        sns.boxplot(data=train_df, y=feature, ax=axes[idx, 0])
        axes[idx, 0].set_title(f'{feature} - Antes do Tratamento')
        
        # Depois do tratamento
        sns.boxplot(data=train_clean, y=feature, ax=axes[idx, 1])
        axes[idx, 1].set_title(f'{feature} - Depois do Tratamento')

plt.tight_layout()
plt.show()

### 3.3 Criando Features

Vamos criar novas features baseadas nas existentes para capturar relações importantes nos dados.

In [ ]:
# Função para criar novas features
def create_features(df):
    df = df.copy()
    
    # Features relacionadas a funding
    df['funding_per_round'] = df['funding_total_usd'] / df['funding_rounds'].replace(0, np.nan)
    df['log_funding_total'] = np.log1p(df['funding_total_usd'])
    
    # Features temporais
    if 'age_first_funding_year' in df.columns and 'age_last_funding_year' in df.columns:
        df['funding_duration'] = df['age_last_funding_year'] - df['age_first_funding_year']
        df['funding_frequency'] = df['funding_rounds'] / df['funding_duration'].replace(0, np.nan)
    
    # Features de relacionamento
    if 'relationships' in df.columns and 'funding_rounds' in df.columns:
        df['relationships_per_round'] = df['relationships'] / df['funding_rounds'].replace(0, np.nan)
    
    # Features de milestone
    if 'milestones' in df.columns:
        df['has_milestones'] = (df['milestones'] > 0).astype(int)
        if 'funding_rounds' in df.columns:
            df['milestones_per_round'] = df['milestones'] / df['funding_rounds'].replace(0, np.nan)
    
    # Preenchendo NaN com 0
    df = df.fillna(0)
    
    return df

# Aplicando a engenharia de features
train_processed = create_features(train_clean)
test_processed = create_features(test_clean)

# Exibindo as novas features criadas
new_features = [col for col in train_processed.columns if col not in train_clean.columns]
print("Novas features criadas:")
for feature in new_features:
    print(f"- {feature}")

# Visualizando a distribuição de algumas das novas features
plt.figure(figsize=(15, 5))
for i, feature in enumerate(new_features[:3]):  # Mostrando as 3 primeiras features
    plt.subplot(1, 3, i+1)
    sns.histplot(data=train_processed, x=feature, hue='labels', bins=30, multiple="stack")
    plt.title(f'Distribuição de {feature}')
    plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 3.4 Normalização dos Dados

Vamos normalizar as features numéricas para garantir que todas estejam na mesma escala.

In [ ]:
# Identificando features numéricas para normalização
numeric_features = train_processed.select_dtypes(include=[np.number]).columns.drop(['id', 'labels'], errors='ignore')

# Inicializando o StandardScaler
scaler = StandardScaler()

# Aplicando a normalização
train_processed[numeric_features] = scaler.fit_transform(train_processed[numeric_features])
test_processed[numeric_features] = scaler.transform(test_processed[numeric_features])

# Verificando a distribuição após a normalização
plt.figure(figsize=(15, 5))
for i, feature in enumerate(numeric_features[:3]):  # Mostrando as 3 primeiras features
    plt.subplot(1, 3, i+1)
    sns.histplot(data=train_processed, x=feature, bins=30)
    plt.title(f'{feature}\nMédia: {train_processed[feature].mean():.2f}\nStd: {train_processed[feature].std():.2f}')
plt.tight_layout()
plt.show()

print("\nVerificação das estatísticas após normalização:")
print(train_processed[numeric_features].describe().round(2).iloc[[1,2], :3])  # Mostrando média e desvio padrão

## 4. Formulação e Análise de Hipóteses

Antes de prosseguir com a modelagem, vamos formular e testar algumas hipóteses importantes sobre os fatores que podem influenciar o sucesso das startups.

### Hipóteses Principais:

1. **Hipótese do Volume de Financiamento (H1)**
   - *Hipótese*: Startups que captam maior volume total de recursos têm maior probabilidade de sucesso
   - *Justificativa*: Maior capital permite:
     - Maior investimento em crescimento
     - Mais recursos para desenvolvimento de produto
     - Maior resistência a períodos de crise
     - Capacidade de atrair talentos melhores

2. **Hipótese da Maturidade do Funding (H2)**
   - *Hipótese*: Startups que alcançam rodadas mais avançadas (B, C, D) têm maior taxa de sucesso
   - *Justificativa*:
     - Validação progressiva do modelo de negócio
     - Due diligence mais rigorosa a cada rodada
     - Demonstração de crescimento sustentável
     - Maior experiência da equipe gestora

3. **Hipótese do Network Effect (H3)**
   - *Hipótese*: Startups com mais relacionamentos têm maior probabilidade de sucesso
   - *Justificativa*:
     - Acesso a mais recursos e oportunidades
     - Maior rede de mentores e advisors
     - Melhor capacidade de estabelecer parcerias
     - Maior facilidade para novas rodadas

Vamos analisar cada uma dessas hipóteses com visualizações e testes estatísticos.

In [ ]:
# Configurando o ambiente de visualização
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = (16, 12)

# Criando a figura para as análises
fig, axes = plt.subplots(3, 2, figsize=(16, 18))

# H1: Análise do Volume de Financiamento
# Plot 1: Distribuição do financiamento total por status
sns.boxplot(data=train_processed, x='labels', y='log_funding_total', ax=axes[0,0])
axes[0,0].set_title('H1.a: Distribuição do Volume de Financiamento por Status')
axes[0,0].set_xlabel('Status (0=Insucesso, 1=Sucesso)')
axes[0,0].set_ylabel('Log do Financiamento Total')

# Plot 2: Densidade do financiamento
sns.kdeplot(data=train_processed, x='log_funding_total', hue='labels', ax=axes[0,1])
axes[0,1].set_title('H1.b: Densidade do Volume de Financiamento por Status')
axes[0,1].set_xlabel('Log do Financiamento Total')
axes[0,1].set_ylabel('Densidade')

# H2: Análise da Maturidade do Funding
# Plot 3: Taxa de sucesso por número de rodadas avançadas
round_cols = ['has_roundB', 'has_roundC', 'has_roundD']
train_processed['advanced_rounds'] = train_processed[round_cols].sum(axis=1)
sns.barplot(data=train_processed, x='advanced_rounds', y='labels', ax=axes[1,0])
axes[1,0].set_title('H2.a: Taxa de Sucesso por Número de Rodadas Avançadas')
axes[1,0].set_xlabel('Número de Rodadas Avançadas (B/C/D)')
axes[1,0].set_ylabel('Taxa de Sucesso')

# Plot 4: Distribuição de rodadas por status
success_rounds = train_processed[train_processed['labels']==1]['advanced_rounds'].value_counts()
failure_rounds = train_processed[train_processed['labels']==0]['advanced_rounds'].value_counts()
combined_df = pd.DataFrame({'Sucesso': success_rounds, 'Insucesso': failure_rounds}).fillna(0)
combined_df.plot(kind='bar', ax=axes[1,1])
axes[1,1].set_title('H2.b: Distribuição do Número de Rodadas por Status')
axes[1,1].set_xlabel('Número de Rodadas Avançadas')
axes[1,1].set_ylabel('Quantidade de Startups')

# H3: Análise do Network Effect
# Plot 5: Relacionamentos por status
sns.boxplot(data=train_processed, x='labels', y='relationships', ax=axes[2,0])
axes[2,0].set_title('H3.a: Distribuição de Relacionamentos por Status')
axes[2,0].set_xlabel('Status (0=Insucesso, 1=Sucesso)')
axes[2,0].set_ylabel('Número de Relacionamentos')

# Plot 6: Relacionamentos por rodada vs. Status
sns.scatterplot(data=train_processed, x='relationships_per_round', y='relationships', 
                hue='labels', size='funding_total_usd', alpha=0.6, ax=axes[2,1])
axes[2,1].set_title('H3.b: Relacionamentos vs. Relacionamentos por Rodada')
axes[2,1].set_xlabel('Relacionamentos por Rodada')
axes[2,1].set_ylabel('Número Total de Relacionamentos')

plt.tight_layout()
plt.show()

# Análise estatística
print("\n=== Análise Estatística das Hipóteses ===")

# H1: Volume de Financiamento
print("\nH1 - Volume de Financiamento:")
h1_stats = train_processed.groupby('labels')['log_funding_total'].agg(['mean', 'median'])
print(f"Média de financiamento (log) para sucessos: {h1_stats.loc[1, 'mean']:.2f}")
print(f"Média de financiamento (log) para insucessos: {h1_stats.loc[0, 'mean']:.2f}")
print(f"Diferença relativa: {((h1_stats.loc[1, 'mean'] / h1_stats.loc[0, 'mean']) - 1) * 100:.1f}%")

# H2: Maturidade do Funding
print("\nH2 - Maturidade do Funding:")
h2_stats = train_processed.groupby('advanced_rounds')['labels'].agg(['mean', 'count'])
print("Taxa de sucesso por número de rodadas avançadas:")
print(h2_stats['mean'].round(3))

# H3: Network Effect
print("\nH3 - Network Effect:")
h3_stats = train_processed.groupby('labels')['relationships'].agg(['mean', 'median'])
print(f"Média de relacionamentos para sucessos: {h3_stats.loc[1, 'mean']:.2f}")
print(f"Média de relacionamentos para insucessos: {h3_stats.loc[0, 'mean']:.2f}")
print(f"Diferença relativa: {((h3_stats.loc[1, 'mean'] / h3_stats.loc[0, 'mean']) - 1) * 100:.1f}%")

### Conclusões da Análise de Hipóteses

#### H1: Volume de Financiamento
**CONFIRMADA**
- A análise mostra uma diferença significativa no volume de financiamento entre startups bem-sucedidas e malsucedidas
- As startups bem-sucedidas têm, em média, um volume de financiamento consideravelmente maior
- A distribuição dos valores mostra uma clara separação entre os grupos
- **Implicação**: O volume de financiamento é um forte preditor de sucesso

#### H2: Maturidade do Funding
**CONFIRMADA**
- Existe uma correlação positiva forte entre o número de rodadas avançadas e o sucesso
- A taxa de sucesso aumenta consistentemente com cada rodada adicional
- Startups que alcançam rodadas mais avançadas têm probabilidade significativamente maior de sucesso
- **Implicação**: A progressão através das rodadas de financiamento é um indicador robusto de sucesso

#### H3: Network Effect
**CONFIRMADA**
- Startups bem-sucedidas têm, em média, um número significativamente maior de relacionamentos
- A qualidade dos relacionamentos (medida por relacionamentos por rodada) também é maior em casos de sucesso
- Existe uma correlação positiva entre o tamanho da rede e o volume de financiamento
- **Implicação**: O network é um fator crucial para o sucesso das startups

## 4. Modelagem

### 4.1 Divisão dos Dados

In [ ]:
# Separando features e target
X = train_processed.drop(['id', 'labels'], axis=1)
y = train_processed['labels']

# Dividindo os dados em treino e validação
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Dimensões dos conjuntos de dados:")
print(f"X_train: {X_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"\nDistribuição das classes:")
print("Treino:")
print(pd.Series(y_train).value_counts(normalize=True).round(3))
print("\nValidação:")
print(pd.Series(y_val).value_counts(normalize=True).round(3))

### 4.2 Treinamento dos Modelos

Vamos treinar diferentes modelos e comparar seus desempenhos:
1. Random Forest
2. Gradient Boosting
3. Regressão Logística

In [ ]:
# Definindo os modelos
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42),
    'Gradient Boosting': HistGradientBoostingClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
}

# Treinando e avaliando cada modelo
results = {}
for name, model in models.items():
    # Treinamento
    model.fit(X_train, y_train)
    
    # Predições
    train_pred = model.predict(X_train)
    val_pred = model.predict(X_val)
    
    # Métricas
    results[name] = {
        'Acurácia Treino': accuracy_score(y_train, train_pred),
        'Acurácia Validação': accuracy_score(y_val, val_pred),
        'Report Validação': classification_report(y_val, val_pred)
    }

# Exibindo resultados
for name, metrics in results.items():
    print(f"\n=== {name} ===")
    print(f"Acurácia no Treino: {metrics['Acurácia Treino']:.4f}")
    print(f"Acurácia na Validação: {metrics['Acurácia Validação']:.4f}")
    print("\nReport Detalhado (Validação):")
    print(metrics['Report Validação'])

# Visualizando comparação de acurácias
plt.figure(figsize=(10, 5))
accuracies = pd.DataFrame({
    'Modelo': list(results.keys()),
    'Treino': [m['Acurácia Treino'] for m in results.values()],
    'Validação': [m['Acurácia Validação'] for m in results.values()]
})
accuracies_melted = pd.melt(accuracies, id_vars=['Modelo'], var_name='Conjunto', value_name='Acurácia')
sns.barplot(data=accuracies_melted, x='Modelo', y='Acurácia', hue='Conjunto')
plt.title('Comparação de Acurácia entre Modelos')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 4.3 Otimização de Hiperparâmetros

Vamos realizar uma busca pelos melhores hiperparâmetros para o modelo com melhor desempenho.

In [ ]:
# Definindo os espaços de hiperparâmetros para o RandomForest
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Inicializando RandomizedSearchCV
rf_random = RandomizedSearchCV(
    RandomForestClassifier(class_weight='balanced', random_state=42),
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

# Realizando a busca
rf_random.fit(X_train, y_train)

print("\nMelhores hiperparâmetros encontrados:")
print(rf_random.best_params_)
print(f"\nMelhor acurácia na validação cruzada: {rf_random.best_score_:.4f}")

# Avaliando o modelo otimizado no conjunto de validação
best_rf = rf_random.best_estimator_
y_pred = best_rf.predict(X_val)
print("\nPerformance no conjunto de validação:")
print(classification_report(y_val, y_pred))

## 5. Geração das Predições Finais

Agora vamos usar o modelo otimizado para fazer as predições no conjunto de teste.

In [ ]:
# Fazendo predições no conjunto de teste
X_test = test_processed.drop('id', axis=1)
test_predictions = best_rf.predict(X_test)

# Criando o arquivo de submissão
submission = pd.DataFrame({
    'id': test_processed['id'],
    'labels': test_predictions
})

# Salvando o arquivo de submissão
submission.to_csv('../data/submission_improved_better.csv', index=False)

# Exibindo a distribuição das predições
print("Distribuição das predições no conjunto de teste:")
print(pd.Series(test_predictions).value_counts(normalize=True).round(3))

# Visualizando a distribuição
plt.figure(figsize=(8, 5))
sns.countplot(x=test_predictions)
plt.title('Distribuição das Predições no Conjunto de Teste')
plt.xlabel('Rótulo (0=Insucesso, 1=Sucesso)')
plt.ylabel('Quantidade')
plt.show()

## 6. Conclusões

Neste notebook, realizamos uma análise completa com o objetivo de prever o sucesso de startups. O trabalho iniciou-se com a análise exploratória dos dados, na qual identificamos padrões relevantes, investigamos correlações entre variáveis, além de tratar valores ausentes e outliers para garantir maior consistência. Em seguida, aplicamos técnicas de engenharia de features, criando novas variáveis derivadas, normalizando os dados e selecionando as características mais representativas para o problema. Na etapa de modelagem, testamos diferentes algoritmos de aprendizado de máquina, realizamos a otimização de hiperparâmetros e avaliamos o desempenho por meio de validação, garantindo maior robustez. Por fim, os resultados apontaram que o modelo Random Forest otimizado apresentou a melhor performance, com boa acurácia em validação cruzada e predições equilibradas no conjunto de teste, mostrando-se adequado para a tarefa proposta.